# Models

This page presents overview of the models, recovery method and results.

## Specification of models 
We made effort to minize the differences between models and to keep features that have previously been shown to improve model fit constant. This in particular concerns two model features:
1. **Starting probability** all models estimate starting probability as a free parameter
2. **Outcome sensitivity** all used models have separate parameter for learning from shocks versus no-shocks

### Rescorla-Wagner (RW3)
### Pearce-Hall (PH5)
### Volatility learner (VOL6) 
### Beta State Inference Model (B-SIM)







## Model recovery 

In [ ]:
addpath('/data/drive/DPhil/THESIS/Cross_study_analyses/learning-repo/scripts')
clear all
ca=1;
comp = 'onx';
pX_analysis_init; %wipes the entire workspace

subIDs ={};
for i =1:50; subIDs{i} = ['artif_sub_' num2str(i)]; end

%data
model_data = {'cff_RW1', 'cff_RW3', 'cff_PH_4', 'cff_PH_5', 'cff_betastsw_v2g_c'};
model_conf = {'cff_RW1', 'cff_RW3', 'cff_PH_4', 'cff_PH_5', 'cff_betastsw_v2g_c'};
labels_short = {'RW2', 'RW3', 'PH4', 'PH5', 'B-SIM'};

for mdata = 1:numel(model_data)
    
    eval(model_data{mdata});
    data_folder = [output_folder 'models/artificial_model_data/' spec.models{1}];
    model_fitting_to = spec.models{1};
    disp(['Data set: ' model_fitting_to ]);
    clear('spec');
      labels = {};
      
    for i = 1:numel(model_conf); 
        eval(model_conf{i}); spec.noruns = 50;
        labels{i}= spec.models{1};
        model_str = [spec.models{1} '_fit_' num2str(spec.noruns) '_'  spec.err.obsFnc  'fto' spec.err.var '_full_' spec.algorithm '_' spec.error_type];
        M{i}=load([data_folder '/'  model_str '.mat']);
      %  for c = 1:3; M{i}.LR_DATA{c}(id, :) =[]; M{i}.LR_DATA{c}= [M{i}.LR_DATA{c} SData]; end
    end

    method = 'BIC';
    AIC ={};
   
    pal = [ 67 188 205; 234 53 70; 249 200 14; 102 46 155;   12 250 120; 248 102 36; 30 30 240]./255;
    for c = 1:3
        AIC{c}=table;
       
        for m = 1:numel(model_conf)
            AIC{c}.(['m' num2str(m) '_AIC']) = M{m}.LR_DATA{c}.(method);
            
        end
    end
    blocks = {'60/40', '75/25', '90/10'};
    
    f=figure;
    for c=1:3
            subplot(1,3,c);

            data = AIC{c};
      
            mens =[];
            for m = 1:numel(model_conf)
                me = data.(['m' num2str(m) '_AIC']);
                scatter(repmat(m, length(me),1)+randn(length(me),1)*0.03, me, 20, pal(m,:), 'filled', 'MarkerEdgeColor',"none", 'MarkerFaceAlpha', 0.2 );
                hold on;
                plot_mse_pretty(m+0.25, me, pal(m,:), 'mean', 'se', 'off');
                %plot_mse_pretty(m+0.25, me, pal(m,:), 'med', 'perc');
                mens(m) = nanmean(me);
                
            end
            minn = find(mens==min(mens));
            plot([0.5 numel(model_conf)+0.5], [mens(minn) mens(minn)], 'Color', pal(minn,:), 'LineWidth', 2);
            
            
            title({['True model: ' model_fitting_to],['Cue ' num2str(c)  ]});
            t=gca;
            t.XTick = 1:numel(model_conf);
    
            t.XTickLabelRotation = 30;
            ylabel(method);
            t.XTickLabel=labels_short;
            
            if mdata == 3 & c==2
                ylim([-130 40]);
                text(1, -110, 'True model');
                plot([0.2 0.8], repmat(-110, 1,2), 'Color',  pal(mdata,:), 'LineWidth', 4);
                text(1, -120, 'Winning model');
                plot([0.2 0.8], repmat(-120,1,2), 'Color',  pal(minn,:), 'LineWidth', 4);
            else
                ylim([mean(mens)-70 mean(mens)+70]);
                text(1, mean(mens)-50, 'True model');
                plot([0.2 0.8], repmat(mean(mens)-50,1,2), 'Color',  pal(mdata,:), 'LineWidth', 4);
                text(1, mean(mens)-60, 'Winning model');
                plot([0.2 0.8], repmat(mean(mens)-60,1,2), 'Color',  pal(minn,:), 'LineWidth', 4);
            end
            
            

        end
        %f.Position(3) = 750;
        %saveas(f, [root_path 'learning-repo/docs/img/model_rec/' model_fitting_to '.pdf']);
        f.Position(3) = 1000;
        %saveas(f, [root_path 'learning-repo/docs/img/model_rec/' model_fitting_to '.png']);
end